In [ ]:
import pandas as pd
import numpy as np
import os

from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

: 

### Support functions

In [ ]:
# Function that resamples trajectory to have M evenly spaced points
def resample_trajectory(trajectory, M):
    N = len(trajectory)
    seg_len = path_length(trajectory) / M
    g = {0: trajectory[0]}
    i = 0
    k = 1
    alpha = 0.0
    beta = seg_len
    
    while (i < N) and (k < M):
        x0, y0 = trajectory[i]
        x1, y1 = trajectory[(i+1) % N]
        d = ((x1 - x0)**2 + (y1 - y0)**2)**0.5
        
        while (beta <= alpha + d) and (k < M):
            xk = x0 + (beta - alpha) * (x1 - x0) / d
            yk = y0 + (beta - alpha) * (y1 - y0) / d
            g[k] = (xk, yk)
            k += 1
            beta += seg_len
            
        alpha += d
        i += 1
        
    return g
    
# Function that computes the total length of the trajectory
def path_length(trajectory):
    N = len(trajectory)
    length = 0.0
    for i in range(0,N):
        x0, y0 = trajectory[i]
        x1, y1 = trajectory[(i+1) % N]
        length += ((x1 - x0)**2 + (y1 - y0)**2)**0.5
    return length

: 

In [ ]:
# Function that computes the Fourier Descriptor Transform
def fourier_descriptor_transform(g):
    N = len(g)
    z = np.array([complex(x, y) for x, y in g.values()])
    Z = np.fft.fft(z) / N
    return Z

# Function that computes the Inverse Fourier Descriptor Transform
def inverse_fourier_descriptor_transform(Z):
    N = len(Z)
    z = np.fft.ifft(Z * N)
    g_reconstructed = {m: (z[m].real, z[m].imag) for m in range(N)}
    return g_reconstructed

: 

### Transforming drawings to Fourier descriptors

**Single file**

In [ ]:
directory = "drawings/normal/"
filepath = directory + "stisnjena3.csv"
df = pd.read_csv(filepath)
df.drop_duplicates(subset=['x', 'y'], keep='first', inplace=True)
df['x'] = df['x'] - df['x'].iloc[0]
df['y'] = df['y'] - df['y'].iloc[0]

M = 100 # Number of points to resample to
g = resample_trajectory(list(zip(df['x'], df['y'])), M)

: 

In [ ]:
Z = fourier_descriptor_transform(g)
g_reconst = inverse_fourier_descriptor_transform(Z)

: 

In [ ]:
list(g_reconst.values())[:][0]

: 

In [ ]:
g_rec_df = pd.DataFrame.from_dict(g_reconst, orient='index', columns=['x', 'y'])
g_rec_df.to_csv(directory + "stisn_rec_" + str(M) + '_test' + '.csv', index=False)

: 

**Multiple files**

In [ ]:
M = 500

directory = "drawings/long/"
# filenames = ["spirala", "stisnjena", "nazobcena", "flat"]
filenames = ["l_spiral", "l_spiky", "l_tight", "l_flat"]

filepaths = [directory + name + f"{i}.csv" for name in filenames for i in range(1,6)]

: 

In [ ]:
df_general = pd.DataFrame()

for filepath in filepaths:
    df = pd.read_csv(filepath)
    df.drop_duplicates(subset=['x', 'y'], keep='first', inplace=True)
    df['x'] = df['x'] - df['x'].iloc[0]
    df['y'] = df['y'] - df['y'].iloc[0]

    g = resample_trajectory(list(zip(df['x'], df['y'])), M)
    
    Z = fourier_descriptor_transform(g)
    g_reconst = inverse_fourier_descriptor_transform(Z)

    g_rec_df = pd.DataFrame.from_dict(g_reconst, orient='index', columns=['x', 'y'])
    g_rec_df['dft_real'] = Z.real
    g_rec_df['dft_imag'] = Z.imag
    g_rec_df['original_file'] = os.path.basename(filepath)
    df_general = pd.concat([df_general, g_rec_df], ignore_index=True)

: 

In [ ]:
df_general.to_csv(directory + "l_all_dft_reconstructed_" + str(M) + '.csv', index=False)

: 

### PCA Analysis

**Option 1 - stacking Re and Im parts**

In [ ]:
filepath_pca = "drawings/long/l_all_dft_reconstructed_500.csv"
df_pca = pd.read_csv(filepath_pca)

df_pca.head()

: 

In [ ]:
# Combining vectors per file and stacking the real and imaginary parts
grouped = df_pca.groupby('original_file')

features_pca = []
features_names = []

for name, group in grouped:
    real_parts = group['dft_real'].values
    imag_parts = group['dft_imag'].values
    feature_vector = np.concatenate([real_parts, imag_parts])
    features_pca.append(feature_vector)
    features_names.append(name)

features_pca = np.array(features_pca)
print("features_pca shape: " + str(features_pca.shape))

: 

In [ ]:
pca = PCA(n_components=2)
X_pca = pca.fit_transform(features_pca)
print("X_pca shape: " + str(X_pca.shape))

: 

In [ ]:
labels = 5*[0] + 5*[1] + 5*[2] + 5*[3]
labels_dic = {0: 'Spiral', 1: 'Spiky', 2: 'Tight', 3: 'Flat'}

plt.figure(figsize=(8,6))
scatter = plt.scatter(X_pca[:,0], X_pca[:,1], c=labels, cmap='viridis', s=50)
plt.title('PCA of Fourier Descriptors')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')

# Point annotation
for i, name in enumerate(features_names):
    plt.annotate(name, (X_pca[i,0]+2, X_pca[i,1]+5), fontsize=6)

for i in np.unique(labels):
    plt.scatter([], [], color=scatter.cmap(scatter.norm(i)), label=f"{labels_dic[i]}")
plt.legend()
plt.grid()
plt.show()

: 

In [ ]:
loadings = pd.DataFrame(
    pca.components_.T,
    columns=[f"PC{i+1}" for i in range(pca.n_components_)],
    #index=feature_names  # list of your original feature names
)

print("Explained variance ratio:", pca.explained_variance_ratio_)
display(loadings.head(10))

: 

In [ ]:
n_components = loadings.shape[1]
n_features = loadings.shape[0]
half = n_features // 2  # split point between real and imaginary

x = np.arange(half)  # x-axis for both halves

fig, axes = plt.subplots(n_components, 1, figsize=(10, 3 * n_components), sharex=True)

if n_components == 1:
    axes = [axes]  # ensure iterable

for i, ax in enumerate(axes):
    pc_name = loadings.columns[i]
    
    # Split loadings
    real_load = loadings.iloc[:half, i]
    imag_load = loadings.iloc[half:, i]
    
    # Plot both halves
    ax.bar(x, real_load, color='royalblue', alpha=0.7, label='Real')
    ax.bar(x, imag_load, color='darkorange', alpha=0.7, label='Imag', width=0.4)
    
    ax.set_title(f"Feature Loadings for {pc_name}")
    ax.set_ylabel("Weight")
    ax.grid(True, axis='y', linestyle='--', alpha=0.4)
    ax.legend()

axes[-1].set_xlabel("Feature index (0–N/2−1)")
plt.tight_layout()
plt.show()

: 

Plot spiral of interest:

In [ ]:
spiral = "l_tight2.csv"
spiral_path = directory + spiral

df_spiral = pd.read_csv(spiral_path)
df_spiral.drop_duplicates(subset=['x', 'y'], keep='first', inplace=True)
df_spiral['x'] = df_spiral['x'] - df_spiral['x'].iloc[0]
df_spiral['y'] = df_spiral['y'] - df_spiral['y'].iloc[0]

num_of_desc = features_pca.shape[1] // 2

spiral_idx = features_names.index(spiral)
spiralZ = features_pca[spiral_idx][:num_of_desc] + 1j * features_pca[spiral_idx][num_of_desc:]
g_spiral_reconst = inverse_fourier_descriptor_transform(spiralZ)

ax, fig = plt.subplots(1, 2, figsize=(12,6))
fig[0].set_title(f'Original Spiral Drawing (M={num_of_desc})')
fig[0].plot(df_spiral['x'], df_spiral['y'])
fig[1].set_title(f'Reconstructed Spiral Drawing from DFT(M={num_of_desc})')
fig[1].plot(*zip(*g_spiral_reconst.values()))
plt.show()

: 

**Option 2 - rotation invariant descriptors**

In [ ]:
df_ri = pd.read_csv(filepath_pca)
df_ri.head()

: 

In [ ]:
# Combining vectors per file and stacking the real and imaginary parts
grouped = df_ri.groupby('original_file')

features_ri = []

for _, group in grouped:
    # Normalize for rotation invariance
    Z = group['dft_real'].values + 1j * group['dft_imag'].values

    # Find first significant harmonic (nonzero magnitude)
    k0 = np.argmax(np.abs(Z[1:]) > 1e-8) + 1  # skip DC term Z[0]
    phase = np.angle(Z[k0])

    Zi = Z * np.exp(-1j * phase)

    
    real_parts = Zi.real
    imag_parts = Zi.imag
    feature_vector = np.concatenate([real_parts, imag_parts])
    features_ri.append(feature_vector)

features_ri = np.array(features_ri)
print("features_ri shape: " + str(features_ri.shape))

: 

In [ ]:
pca_ri = PCA(n_components=2)
X_ri = pca_ri.fit_transform(features_ri)
print("X_ri shape: " + str(X_ri.shape))

: 

In [ ]:
plt.figure(figsize=(8, 6))

# Main scatter plot
scatter = plt.scatter(
    X_ri[:, 0], X_ri[:, 1],
    c=labels, cmap='viridis', s=50, alpha=0.8, edgecolors='k', linewidths=0.3
)

plt.title('PCA of Fourier Descriptors – rotation invariant')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')

# Optional: annotate each point (only if there aren’t too many)
for i, name in enumerate(features_names):
    plt.annotate(
        name,
        (X_ri[i, 0] + 2, X_ri[i, 1] + 5),
        fontsize=6,
        alpha=0.8
    )

# Legend based on cluster colors
handles = [
    plt.Line2D(
        [], [], marker='o', color='w',
        markerfacecolor=scatter.cmap(scatter.norm(i)),
        label=labels_dic[i], markersize=8
    )
    for i in np.unique(labels)
]
plt.legend(handles=handles, title="Clusters", frameon=True)
plt.axis('equal')
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

: 

In [ ]:
loadings_ri = pd.DataFrame(
    pca_ri.components_.T,
    columns=[f"PC{i+1}" for i in range(pca_ri.n_components_)],
    #index=feature_names  # list of your original feature names
)

print("Explained variance ratio:", pca_ri.explained_variance_ratio_)
display(loadings_ri.head(10))

: 

### tSNE Analysis

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import numpy as np

# --- Compute t-SNE ---
tsne = TSNE(
    n_components=2,       
    perplexity=4,          # small if you have few spirals
    learning_rate=200,    
    random_state=42
)
X_embedded = tsne.fit_transform(features_ri)

# --- Visualize ---
plt.figure(figsize=(8,6))
scatter = plt.scatter(
    X_embedded[:, 0], 
    X_embedded[:, 1], 
    c=labels,            # color points by cluster label
    cmap='tab10',        # consistent and qualitative colormap
    s=30, 
    alpha=0.8
)

for i, name in enumerate(features_names):
    plt.annotate(
        name,
        (X_embedded[i, 0] + 2, X_embedded[i, 1] + 5),
        fontsize=6,
        alpha=0.7
    )

# --- Legend ---
for i in np.unique(labels):
    plt.scatter([], [], color=scatter.cmap(scatter.norm(i)), label=labels_dic[i])
plt.legend(title="Spiral Type", bbox_to_anchor=(1.05, 1), loc='upper left')

plt.title("t-SNE of Spiral Feature Vectors")
plt.xlabel("t-SNE 1")
plt.ylabel("t-SNE 2")
plt.tight_layout()
plt.show()

: 

In [ ]:
Z

: 

### Interactive plot

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

# --- Load spiral and its Fourier descriptors ---
spiral = "l_spiky2.csv"
df = grouped.get_group(spiral)
x_real = df['x'].values
y_real = df['y'].values

# Reconstruct complex Fourier descriptors
Z = df['dft_real'].values + 1j * df['dft_imag'].values
N = len(Z)

# Choose which two descriptors to manipulate
idx1, idx2 = 1, 2

# Get magnitude of descriptors for slider range
mag1, mag2 = np.abs(Z[idx1]), np.abs(Z[idx2])

def update_descriptor(re1=0.0, im1=0.0, re2=0.0, im2=0.0):
    Z_mod = Z.copy()
    
    # Apply slider values
    Z_mod[idx1] = re1 + 1j * im1
    Z_mod[idx2] = re2 + 1j * im2
    
    # Reconstruct
    recon = inverse_fourier_descriptor_transform(Z_mod)
    
    plt.figure(figsize=(6,6))
    plt.plot(x_real, y_real, color='gray', alpha=0.3, lw=2, label='Original')
    plt.plot([t[0] for t in recon.values()], [t[1] for t in recon.values()], color='blue', lw=2, label='Reconstruction')
    plt.axis('equal')
    plt.legend()
    plt.show()

# --- Sliders with ranges proportional to descriptor magnitudes ---
interact(
    update_descriptor,
    re1=FloatSlider(min=-2*mag1, max=2*mag1, step=mag1/50, value=Z[idx1].real, description=f'Re(Z{idx1})'),
    im1=FloatSlider(min=-2*mag1, max=2*mag1, step=mag1/50, value=Z[idx1].imag, description=f'Im(Z{idx1})'),
    re2=FloatSlider(min=-2*mag2, max=2*mag2, step=mag2/50, value=Z[idx2].real, description=f'Re(Z{idx2})'),
    im2=FloatSlider(min=-2*mag2, max=2*mag2, step=mag2/50, value=Z[idx2].imag, description=f'Im(Z{idx2})')
)

: 

In [ ]:
from ipywidgets import interact, FloatSlider

# --- Load spiral and its Fourier descriptors ---
spiral = "l_spiky3.csv"
df = grouped.get_group(spiral)
x_real = df['x'].values
y_real = df['y'].values

# Reconstruct complex Fourier descriptors
Z = df['dft_real'].values + 1j * df['dft_imag'].values
N = len(Z)

PCA1_w = loadings_ri.iloc[:,0].values
PCA2_w = loadings_ri.iloc[:,1].values

def update_descriptor(c1=0.0, c2=0.0):
    Z_mod = Z.copy()
    
    # Apply slider values
    Z_mod += c1 * PCA1_w[:N] + c1 * PCA1_w[N:] * 1j + c2 * PCA2_w[:N] + c2 * PCA2_w[N:] * 1j
    
    # Reconstruct
    recon = inverse_fourier_descriptor_transform(Z_mod)
    
    plt.figure(figsize=(6,6))
    plt.plot(x_real, y_real, color='gray', alpha=0.3, lw=2, label='Original')
    plt.plot([t[0] for t in recon.values()], [t[1] for t in recon.values()], color='blue', lw=2, label='Reconstruction')
    plt.axis('equal')
    plt.legend()
    plt.show()

# --- Sliders with ranges proportional to descriptor magnitudes ---
interact(
    update_descriptor,
    c1=FloatSlider(min=-20, max=20, step=1/5, value=0, description=f'PCA1'),
    c2=FloatSlider(min=-20, max=20, step=1/5, value=0, description=f'PCA2'),
)

: 